# TRANSFER LEARNING

Transfer Learning 
Transfer Learning bir şeyi bir yerden bir yere taşımak. Öğrenilen bir şeyi taşımak gibi düşünülebilir. Deep learning’de transfer learning kavramı nedir? 

Bir CNN yapısı düşünelim. Bu CNN yapısında bir input var ve bu input layer’lardan geçiyor. Ardından bir output üretilir. Peki, bu CNN yapısını eğittiğimiz zaman neleri öğreniyoruz. 1. Filtreler 2. Weight değerlerini(Fully Connected değerleri) öğrenmiş oluyoruz. Öncesinde binlerce class’ı olan bir dataset eğitip ardından öğrendiğimiz bu weight değerlerini elimizde tutup ardından başka class’lara sahip olan bir veriseti için kullanabilmek özelliği “Transfer Learning” denir. “Low Level Seviyede” öğrenilmiş weightler ile farklı bir datasetini eğitmeyi mümkün kılmaktır. Bu olaya “PreTrained” denir. Yeni class için “fine funing” denir. 
Farklı bir şekilde elde almak istersek bir verisetinin kedi ve köpek class’larını sınıflandrmak için kullandığımızı ve eğittiğimizi düşünürsek; ve output değerlerini elde ettiğimizi varsayarsak, ekstra bir class daha(mesela hamster) classify etmek istersek binary classification(sigmoid function) adımına kadar olan kısmını kullanarak farklı bir hayvanı classify etmek için farklı bir “aktivasyon fonksiyonu” kullanarak aynı weight değerleri ile eğitilebilir. Yani eğitim adımında işimize yarayacak kısmını kullanmaktır. 

Transfer Learning Avantajları: Zaman- HyperParametre Tuning- (Resimlerin low level ve neredeyse middle level feature’ları aynıdır.)


Transfer Learning Örnekleri :  VGG(keras ile ImageNet sınıfı) ve ResNet

Transfer Learning
Biz Transfer Learning kavramını anlayabilmek için Fruit360 datasetini kullanacağız. 
Öncesinde gerekli kütüphanelerimizi import etmekle işe başlayacağız. Ardından Fruit360 datasetimizin içinde train ve test verisetimizin yoluunu gösterelim.


In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16  #VGG16 modeli import edilmiştir.
import matplotlib.pyplot as plt
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


In [2]:
train_path= "C:/Desktop/fruits-360_dataset/fruits-360/Training"
test_path= "C:/Desktop/fruits-360_dataset/fruits-360/Test"

Şimdi ise glob fonksiyonu yardımıyla sınıf sayımızı öğrenelim.

In [3]:
numberOfClass = len(glob(train_path+"/*"))
numberOfClass

131

#  VGG16 MODELİ KULLANIMI

Önce vgg modelinden bir obje yaratalım.

In [4]:
from keras.applications.vgg16 import VGG16 
vgg = VGG16(include_top = False,input_shape = (100,100,3)) #16 layerdan oluşan VGG modeli

Şimdi VGG16 modelinin özetine bakalım ve bizim datasetimize uyarlamak adımına geçelim.

İçinde varolan tüm layer’ları ve train edilecebilecek görsel sayıları, total parametreleri gibi özellikleri gördük. Şimdi ise incelediğimizde VGG16 modelinin 1000 sınıfı sınıflandırabilme yetkisinin olduğunu gördük fakat bizim class sayımızı sadece 96 adettir. Haliyle “Prediction” layer’ını modelden silip bizim için uygun olabilecek bir Dense Layer ekleyebiliriz. 

In [5]:
print(vgg.summary())


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [6]:
print(type(vgg))


<class 'keras.engine.functional.Functional'>


Öncesinde tüm layer’ları bir liste içine atalım.

In [7]:
vgg_layer_list = vgg.layers
print(vgg_layer_list)

[<keras.engine.input_layer.InputLayer object at 0x0000016FCEF3D4B0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDBBA4F10>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDBBA5180>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000016FDBBA61D0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDBBA5930>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCF93C70>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000016FDCFB7F40>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCFB67A0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCFD3910>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCFD3AF0>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000016FDCFF3610>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCFF2980>, <keras.layers.convolutional.conv2d.Conv2D object at 0x0000016FDCFF0F40>, <keras.layers.convolutional.conv2d

In [8]:
len(vgg_layer_list)


19

Bir layer ekleyeceğimizi belirtmiştik. Şimdi ise layer’ı ekleyelim. Bu adım Sequential() bir şekilde gerçekleşeceğinden ;

In [9]:
model = Sequential()


In [10]:
for i in range (len(vgg_layer_list)-1):
    model.add(vgg_layer_list[i])


Yalnızca son satırı ekleyip kullanacağımız için diğer layer’ların trainable özelliğini kapatıyoruz. 


In [11]:

for layers in model.layers:
    layers.trainable = False


In [12]:

model.add(Flatten()) 
model.add(Dense(256, activation="relu")) 
model.add(Dense(150, activation="relu"))

Yukarıdaki kod ile model'e Dense layer yani output layer ekledik. Output layer'ında belirtmemiz gereken 2 tane özellik var. Birincisi number of class'tır yani kaç tane nörondan oluşacağıdır. Yukarıdaki satırlarda tanımladığımız numberOfClass değişkeninde 131 değeri bulunmaktadır. Aktivasyon fonksiyonu da multiclass için kullandığımız Softmax fonksiyonu olacaktır.

Şimdi modelin katmanlarına bakalım.

In [13]:
model.add(Dense(numberOfClass, activation="softmax"))


In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 25, 25, 256)       2

In [15]:

model.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

# TRAIN

Şimdi train işlemine geçebiliriz.
Öncelikle vgg modelimizin input_shape= 224,224 olduğu için ImageGenerator kullanarak verilen directory de olan yani test ve train verisetlerini 224,224 haline getiriyoruz.


In [21]:
# train  
train_data = ImageDataGenerator().flow_from_directory(train_path,target_size = (100,100))
test_data = ImageDataGenerator().flow_from_directory(test_path,target_size = (100,100))

batch_size = 32

hist = model.fit_generator(train_data,
                           steps_per_epoch=1600//batch_size,
                           epochs= 13,
                           validation_data=test_data,
                           validation_steps= 800//batch_size)


Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


C:\Users\derin\AppData\Local\Temp\ipykernel_3076\2448932319.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_data,


Epoch 1/13
50/50 [==============================] - 63s 1s/step - loss: 12.6166 - accuracy: 0.1531 - val_loss: 3.9664 - val_accuracy: 0.2138
Epoch 2/13
50/50 [==============================] - 78s 2s/step - loss: 3.8819 - accuracy: 0.2463 - val_loss: 3.9982 - val_accuracy: 0.2650
Epoch 3/13
50/50 [==============================] - 77s 2s/step - loss: 3.4086 - accuracy: 0.3381 - val_loss: 4.2431 - val_accuracy: 0.2800
Epoch 4/13
50/50 [==============================] - 74s 1s/step - loss: 2.9974 - accuracy: 0.4100 - val_loss: 3.6780 - val_accuracy: 0.3812
Epoch 5/13
50/50 [==============================] - 74s 1s/step - loss: 2.4199 - accuracy: 0.5144 - val_loss: 3.0134 - val_accuracy: 0.4137
Epoch 6/13
50/50 [==============================] - 75s 2s/step - loss: 2.1416 - accuracy: 0.5650 - val_loss: 2.7077 - val_accuracy: 0.4750
Epoch 7/13
50/50 [==============================] - 75s 2s/step - loss: 1.9087 - accuracy: 0.6044 - val_loss: 2.1486 - val_accuracy: 0.5337
Epoch 8/13
50/50 [=

In [22]:
model.save_weights("deneme.h5")


In [ ]:
#evaluation
#loss
print(hist.history.keys())
plt.plot(hist.history["loss"],label="training loss")
plt.plot(hist.history["val_loss"],label="validation loss")
plt.legend()
plt.show()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
#evaluation
#accuracy
print(hist.history.keys())
plt.plot(hist.history["accuracy"],label="training accuracy")
plt.plot(hist.history["val_accuracy"],label="validation accuracy")
plt.legend()
plt.show()

In [ ]:
#save history
import json, codecs
with open("deneme.json","w") as f:
    json.dump(hist.history,f)

In [ ]:
#Load history
with codecs.open("deneme.json","r",encoding="utf-8") as f:
    n = json.loads(f.read())

In [ ]:
n

In [ ]:
#evaluation
#loss
plt.plot(n["loss"],label="training loss")
plt.plot(n["val_loss"],label="validation loss")
plt.legend()
plt.show()

In [ ]:
#evaluation
#accuracy
plt.plot(n["accuracy"],label="training accuracy")
plt.plot(n["val_accuracy"],label="validation accuracy")
plt.legend()
plt.show()